### Importing the needed libraries

In [1]:
import sys
!{sys.executable} -m pip install imblearn

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler #you can use minmax scaler too
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
from imblearn.ensemble import BalancedBaggingClassifier
#Import other necessary model libraries, for this example, using Logistic Regression

ModuleNotFoundError: No module named 'xgboost'

### Importing the dataset

In [ ]:
train_data = pd.read_csv(r"Data/Preprocessed-Datasets/train-data-encoded.csv", header = 0)
test_data = pd.read_csv(r"../Data/Preprocessed-Datasets/test-data-encoded.csv", header = 0)
validation_data = pd.read_csv(r"../Data/Preprocessed-Datasets/validation-data-encoded.csv", header = 0)

In [3]:
train_data=train_data.drop(['Reservation-id'], axis = 1)
validation_data=validation_data.drop(['Reservation-id'], axis = 1)
test_data=test_data.drop(['Reservation-id'], axis = 1)

NameError: name 'train_data' is not defined

In [ ]:
validation_data.head()

In [ ]:
validation_data.head()

In [ ]:
### Getting names of all the columns
train_data_column_names = train_data.columns

### Seperating the columns of categorical and quantitative data

In [ ]:
data_quan_cols = ['Age','Discount_Rate', 'Room_Rate','Expected_stay_days', 
                        'Reservation_gap', 'Adults', 'Children','Babies']

In [ ]:
data_cat_cols = ['Gender', 'Ethnicity', 'Educational_Level', 'Income', 'Country_region',
                 'Hotel_Type', 'Meal_Type', 'Visted_Previously',
                 'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
                 'Required_Car_Parking', 'Use_Promotion']

In [ ]:
data_cols_plot = ['Age','Discount_Rate', 'Room_Rate','Expected_stay_days','Reservation_gap', 
             'Adults', 'Children','Babies', 'Gender', 'Ethnicity', 'Educational_Level', 
             'Income', 'Country_region','Hotel_Type', 'Meal_Type', 'Visted_Previously',
             'Previous_Cancellations', 'Deposit_type', 'Booking_channel','Required_Car_Parking', 
             'Use_Promotion']

In [ ]:
print(data_cols)

### Scaling the quantitative variables

In [ ]:
train_data_quan = train_data[data_quan_cols]
validation_data_quan = validation_data[data_quan_cols]
test_data_quan = test_data[data_quan_cols]

In [ ]:
sc = StandardScaler()

In [ ]:
train_data_quan_scaled = sc.fit_transform(train_data_quan)
validation_data_quan_scaled = sc.transform(validation_data_quan)
test_data_quan_scaled = sc.transform(test_data_quan)

In [ ]:
train_data_quan_scaled

In [ ]:
type(train_data_quan_scaled)

### Separating the categorical variables

In [ ]:
cols_to_drop = data_quan_cols.copy()
cols_to_drop.append("Reservation_Status")
cols_to_drop

In [ ]:
print(cols_to_drop)

In [ ]:
train_data_cat = train_data.drop(cols_to_drop, axis=1)
validation_data_cat = validation_data.drop(cols_to_drop, axis=1)
test_data_cat = test_data.drop(data_quan_cols, axis=1)
train_data_cat.head()

In [ ]:
train_data_cat_array = train_data_cat.to_numpy()
validation_data_cat_array = validation_data_cat.to_numpy()
test_data_cat_array = test_data_cat.to_numpy()
train_data_cat_array.shape

### Joining the pre-processed data

In [ ]:
train_data_processed = np.concatenate((train_data_quan_scaled, train_data_cat_array), axis = 1)
validation_data_processed = np.concatenate((validation_data_quan_scaled, validation_data_cat_array), axis = 1)
test_data_processed = np.concatenate((test_data_quan_scaled, test_data_cat_array), axis = 1)

In [ ]:
train_data_processed[0]

In [ ]:
test_data_quan_scaled.shape

In [ ]:
test_data_cat_array.shape

In [ ]:
validation_data_processed.shape

### Encoding the labels

In [ ]:
train_y = train_data['Reservation_Status']
validation_y = validation_data['Reservation_Status']

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
train_y = le.fit_transform(train_y)
validation_y = le.transform(validation_y)

In [ ]:
validation_y.shape

In [ ]:
le.classes_

### Under and Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
#0: Cancelled, 1: Check--in , 2:No Show

In [ ]:
#Oversampling
strategy = {0:4134*5, 1:21240, 2:2125*9}
smote = SMOTE(sampling_strategy=strategy) 

X_smote, y_smote = smote.fit_resample(train_data_processed, train_y)
#print(Counter(y_train), Counter(y_smote))

In [ ]:
#NearMiss
from imblearn.under_sampling import TomekLinks #use TomekLinks and NearMiss

strategy = {1: round(21240*0.5), 0: 4134*2, 2: 2125*4}
undersample = TomekLinks()

X_near , y_near = undersample.fit_resample(X_smote,y_smote)

In [ ]:
unique_near = np.bincount(y_near)
print(unique_near)

Use X_near, y_near as points to plot data
Use train_data_column_names for col_names
Making a dataframe out of these

In [ ]:
train_data_processed_df = pd.DataFrame(data = X_near , columns = train_data_column_names)